In [1]:
"""
#Loads a CSV DwC occurrence file that has been augmented with BELS locality strings
"""

'\n#Loads a CSV DwC occurrence file that has been augmented with BELS locality strings\n'

In [2]:
import pandas as pd
import matplotlib

# switch to suppress CSV writing (which is slow)
write_csv = True

In [3]:
#df_occ = pd.read_csv('TORCH-Denton-9f30f07b-e694-4f79-83c3-54fafaa98727.csv', low_memory=False)
#df_occ = pd.read_csv('r_dale_thomas_texas_bels.csv', low_memory=False)
#df_occ = pd.read_csv('r-dale-thomas-all-seinet-texas-bels.csv', low_memory=False)
df_occ = pd.read_csv('torch_bels_locs.csv', low_memory=False, sep='\t')

In [4]:
df_occ.shape

(1384066, 126)

In [5]:
# Drop irrelevant colums
drop_columns= ['higherClassification','kingdom','phylum','class','order','identificationReferences','identificationRemarks','taxonRemarks','identificationQualifier','typeStatus','fieldNumber','eventID','informationWithheld','dataGeneralizations','dynamicProperties','associatedSequences','associatedTaxa','reproductiveCondition','establishmentMeans','lifeStage','sex','individualCount','samplingProtocol','preparations','continent','waterBody','islandGroup','island','rights','rightsHolder','accessRights','recordID','type','license','bibliographicCitation','datasetName','fieldNotes','countryCode','nomenclaturalCode','nomenclaturalStatus','associatedMedia','higherGeography','institutionID','georeferencedDate','datasetID','occurrenceStatus','verbatimLocality','organismID','previousIdentifications','eventTime','eventRemarks','locationAccordingTo','verbatimCoordinateSystem','footprintWKT','earliestEonOrLowestEonothem','earliestEraOrLowestErathem','earliestPeriodOrLowestSystem','earliestEpochOrLowestSeries','earliestAgeOrLowestStage','group','formation','member','identificationVerificationStatus','scientificNameID']
df_occ = df_occ.drop(columns=drop_columns)

In [6]:
df_occ.shape

(1384066, 62)

In [7]:
#list(df_occ.columns)

In [8]:
# Find duplicate records

#bels_matchwithcoords
df_matches = df_occ[df_occ['bels_location_string'].duplicated(keep=False)]
#df_matches = df_denton[df_denton.bels_matchwithcoords

In [9]:
df_matches.shape

(972407, 62)

In [10]:
#list(df_matches.columns)

In [11]:
# Add location ID
# first create a copy to avoid copy of a slice of a DF
df_matches = df_matches.copy()
# https://stackoverflow.com/a/51110197 or https://stackoverflow.com/a/51110205
df_matches['loc_id'] = df_matches.groupby(['bels_location_string']).ngroup()

In [12]:
# Add dup loc count column
#https://stackoverflow.com/a/46768694

df_matches['dup_loc_count'] = df_matches.groupby(['bels_location_string']).transform('size')

In [13]:
#df_matches

In [14]:
if write_csv:
    df_matches.to_csv('TORCH_bels_matches_loc_id.csv')

In [15]:
# find records lacking geo
#bels_decimallatitude
df_nogeo = df_matches[df_matches['decimalLatitude'].isna()]
#TODO add loc ID - https://stackoverflow.com/a/51110205


In [16]:
df_nogeo.shape

(618624, 64)

In [17]:
if write_csv:
    df_nogeo.to_csv('TORCH_bels_matches_nogeo.csv')

In [18]:
# find records with geo
df_geo = df_matches[df_matches['decimalLatitude'].notna()]

In [19]:
df_geo.shape

(353783, 64)

In [20]:
if write_csv:
    df_geo.to_csv('TORCH_bels_matches_geo.csv')

In [21]:
# all dups with and without geocoords
dup_loc_count = df_matches.pivot_table(index = ['bels_location_string'], aggfunc ='size')

In [22]:
dup_loc_count.shape

(149516,)

In [23]:
if write_csv:
    dup_loc_count.to_csv('TORCH_dup_loc_count.csv')

In [24]:
# dups with no goecoords
#dups = df.pivot_table(index = ['Course'], aggfunc ='size') 
dup_loc_count_no_geo = df_nogeo.pivot_table(index = ['bels_location_string'], aggfunc ='size')

In [25]:
dup_loc_count_no_geo.shape

(96572,)

In [26]:
if write_csv:
    dup_loc_count_no_geo.to_csv('TORCH_dup_loc_count_nogeo.csv')

In [27]:
# dups with goecoords
dup_loc_count_geo = df_geo.pivot_table(index = ['bels_location_string'], aggfunc ='size')

In [28]:
dup_loc_count_geo.shape

(70659,)

In [29]:
if write_csv:
    dup_loc_count_geo.to_csv('TORCH_dup_loc_count_geo.csv')

In [82]:
#print(dup_loc_count)

In [83]:
#filter records that mention no additional locality?

In [84]:
#list(df_matches.columns)

In [85]:
# from ChatGPT
#https://chat.openai.com/share/21c561bb-db43-47ee-980a-f88078b8c78b
# Identify duplicate values in a specific column
#
#duplicates = df_matches['bels_matchwithcoords'].duplicated(keep=False)
#duplicates = df_matches['bels_location_string'].duplicated(keep=False)

# Count the number of duplicates for each value
#duplicate_counts = duplicates.groupby(df_matches['bels_matchwithcoords']).sum()
#duplicate_counts = duplicates.groupby(df_matches['bels_location_string']).sum()



In [86]:
# Note needed, done above
# Add a new column to the DataFrame with the number of duplicates
#df_matches['duplicate_count'] = df_matches['bels_location_string'].map(duplicate_counts)

In [87]:
#duplicate_counts.sort_index().cumsum().plot()
#duplicate_counts.set_index('duplicate_count')
#duplicate_counts.sort_index().plot()

In [88]:
#df_matches[['duplicate_count']] = df_matches[['duplicate_count']].apply(pd.to_numeric)

In [89]:
#df_matches.set_index('duplicate_count').cumsum()

In [30]:
#TODO
#Load ids of elite georeferencers - EG
df_georeferencers = pd.read_csv('TORCH-georeferencers_tested.csv', low_memory=False)

#find records with geo that have been done by EGs
#find records without geo that match those done by EGs

In [31]:
# filter records that have been georeferenced by 'elite' vetted georeferencers
df_by_egeo = df_matches[df_matches['georeferencedBy'].isin(df_georeferencers['Username'])]

In [32]:
df_by_egeo.shape

(26134, 64)

In [33]:
if write_csv:
    df_by_egeo.to_csv('TORCH_df_by_egeo.csv')

In [34]:
# find locations that match loc_id of those done by EG
df_loc_match_by_egeo = df_matches[df_matches['loc_id'].isin(df_by_egeo['loc_id'])]

In [35]:
df_loc_match_by_egeo.shape

(46543, 64)

In [36]:
# find matching locs that have not been georeffed
df_loc_match_by_egeo_nogeo = df_loc_match_by_egeo[df_loc_match_by_egeo['decimalLatitude'].isna()]

In [37]:
df_loc_match_by_egeo_nogeo.shape

(13820, 64)

In [38]:
# find matching locs that have been georeffed
df_loc_match_by_egeo_w_geo = df_loc_match_by_egeo[df_loc_match_by_egeo['decimalLatitude'].notna()]

In [39]:
df_loc_match_by_egeo_w_geo.shape

(32723, 64)

In [40]:
df_nogeo.shape

(618624, 64)

In [41]:
# find dup locs without geo but have other dups that have geo
df_nogeo_with_geolocdups = df_nogeo[df_nogeo['loc_id'].isin(df_geo['loc_id'])]

In [42]:
df_nogeo_with_geolocdups.shape

(228143, 64)

In [43]:
if write_csv:
    df_nogeo_with_geolocdups.to_csv('TORCH_nogeo_with_geolocdups.csv')

In [44]:
#TODO 



In [45]:
# Drop records with only the state name in bels string
#df_nogeo_with_geolocdups_nostates = df_nogeo_with_geolocdups[df_nogeo_with_geolocdups[column1] != df[column2]]

df_nogeo_with_geolocdups_nostates = df_nogeo_with_geolocdups[~df_nogeo_with_geolocdups.apply(lambda row: row['stateProvince'].lower() == row['bels_location_string'].lower(), axis=1)]


In [46]:
df_nogeo_with_geolocdups_nostates.shape

(223621, 64)

In [47]:
if write_csv:
    df_nogeo_with_geolocdups_nostates.to_csv('TORCH_nogeo_with_geolocdups_nostates.csv')

In [48]:
# remove state + county matches
df_nogeo_with_geolocdups_nostatecounty = df_nogeo_with_geolocdups_nostates[~df_nogeo_with_geolocdups_nostates.apply(lambda row: (str(row['stateProvince']) + str(row['county'])).lower() == row['bels_location_string'].lower(), axis=1)]

In [49]:
df_nogeo_with_geolocdups_nostatecounty.shape

(156127, 64)

In [50]:
if write_csv:
    df_nogeo_with_geolocdups_nostatecounty.to_csv('TORCH_nogeo_with_geolocdups_nostatecounties.csv')

In [119]:
list[df_nogeo_with_geolocdups_nostatecounty.columns]

list[Index(['id', 'institutionCode', 'collectionCode', 'ownerInstitutionCode',
       'collectionID', 'basisOfRecord', 'occurrenceID', 'catalogNumber',
       'otherCatalogNumbers', 'family', 'scientificName', 'taxonID',
       'scientificNameAuthorship', 'genus', 'subgenus', 'specificEpithet',
       'verbatimTaxonRank', 'infraspecificEpithet', 'taxonRank',
       'identifiedBy', 'dateIdentified', 'recordedBy', 'recordNumber',
       'eventDate', 'year', 'month', 'day', 'startDayOfYear', 'endDayOfYear',
       'verbatimEventDate', 'occurrenceRemarks', 'habitat',
       'associatedOccurrences', 'locationID', 'country', 'stateProvince',
       'county', 'municipality', 'locality', 'locationRemarks',
       'decimalLatitude', 'decimalLongitude', 'geodeticDatum',
       'coordinateUncertaintyInMeters', 'verbatimCoordinates',
       'georeferencedBy', 'georeferenceProtocol', 'georeferenceSources',
       'georeferenceVerificationStatus', 'georeferenceRemarks',
       'minimumElevationInMet

In [51]:
# add counts by county
#df_nogeo_with_geolocdups_nostatecounty
county_summary = df_nogeo_with_geolocdups_nostatecounty.groupby(['stateProvince','county']).agg(
    total_locations=('loc_id', 'count'),
    sum_id_count=('dup_loc_count', 'sum')
)

In [52]:
county_summary

total_locations  sum_id_count
stateProvince county                                   
Oklahoma      Adair                   365         24975
              Alfalfa                 313         19351
              Atoka                   373         10366
              Atoka Co.                 1             2
              Beaver                  158          3603
...                                   ...           ...
Texas         Yoakum                   16           132
              Young                    10            28
              Zapata                  287          4877
              Zapata Co.                1             2
              Zavala                    2             6

[439 rows x 2 columns]

In [53]:
county_summary.to_csv('TORCH_county_summary.csv')

In [54]:
# returning to unfiltered DF for more comprehensive results

#df_nogeo
# remove states only
df_nogeo_nostates = df_nogeo[~df_nogeo.apply(lambda row: row['stateProvince'].lower() == row['bels_location_string'].lower(), axis=1)]
#remove state and county only
df_nogeo_nostatecounty = df_nogeo_nostates[~df_nogeo_nostates.apply(lambda row: (str(row['stateProvince']) + str(row['county'])).lower() == row['bels_location_string'].lower(), axis=1)]

df_nogeo_nostatecounty.shape


(506966, 64)

In [55]:
county_summary_nogeo = df_nogeo_nostatecounty.groupby(['stateProvince','county']).agg(
    #total_locations=('loc_id', 'count'),
    unique_locations=('loc_id', 'nunique'),
    #sum_id_count=('dup_loc_count', 'sum')
    loc_count=('loc_id', 'count')
)
# Reset the index to make 'state' and 'county' regular columns
county_summary_nogeo = county_summary_nogeo.reset_index()
print("\nSummary with reset index:")
print(county_summary_nogeo)


Summary with reset index:
    stateProvince          county  unique_locations  loc_count
0        Oklahoma           Adair               392       1986
1        Oklahoma    Adair County                 2          5
2        Oklahoma         Alfalfa               140        626
3        Oklahoma  Alfalfa County                 5         26
4        Oklahoma        Angelina                 1          2
..            ...             ...               ...        ...
852         Texas        presidio                 1          1
853         Texas           smith                 1          1
854         Texas           starr                 1          1
855         Texas          walker                 2          2
856         Texas         willacy                 1          1

[857 rows x 4 columns]


In [56]:
# Same process but only include BRIT records in Texas
# returning to unfiltered DF for more comprehensive results

#df_nogeo
# Texas collections only
df_torch_texas_nogeo = df_nogeo[df_nogeo['stateProvince'] == 'Texas']
# remove states only
df_torch_texas_nogeo_nostates = df_torch_texas_nogeo[~df_torch_texas_nogeo.apply(lambda row: row['stateProvince'].lower() == row['bels_location_string'].lower(), axis=1)]
#remove state and county only
df_torch_texas_nogeo_nostatecounty = df_torch_texas_nogeo_nostates[~df_torch_texas_nogeo_nostates.apply(lambda row: (str(row['stateProvince']) + str(row['county'])).lower() == row['bels_location_string'].lower(), axis=1)]


In [166]:
df_torch_texas_nogeo_nostatecounty.shape  # old (106414, 64)

(379614, 64)

In [57]:
# BRIT collections only
df_torch_texas_brit_nogeo = df_torch_texas_nogeo_nostatecounty[(df_torch_texas_nogeo_nostatecounty['institutionCode'] == 'BRIT') | (df_torch_texas_nogeo_nostatecounty['institutionCode'] == 'VDB')]

df_torch_texas_brit_nogeo.shape


(77317, 64)

In [58]:
df_torch_texas_brit_nogeo.to_csv('torch_texas_brit_nogeo.csv')

In [59]:
# Summarize BRIT Texas records
brit_texas_county_summary_nogeo = df_torch_texas_brit_nogeo.groupby(['stateProvince','county']).agg(
    #total_locations=('loc_id', 'count'),
    unique_locations=('loc_id', 'nunique'),
    #sum_id_count=('dup_loc_count', 'sum')
    loc_count=('loc_id', 'count'),
    largest_dups = ('dup_loc_count', 'max')
)
# Reset the index to make 'state' and 'county' regular columns
brit_texas_county_summary_nogeo = brit_texas_county_summary_nogeo.reset_index()
print("\nSummary with reset index:")
print(brit_texas_county_summary_nogeo)


Summary with reset index:
    stateProvince    county  unique_locations  loc_count  largest_dups
0           Texas  Anderson               218        824           117
1           Texas   Andrews                 3          4             4
2           Texas  Angelina               137        422            80
3           Texas   Aransas               181        733           295
4           Texas    Archer                39        146            50
..            ...       ...               ...        ...           ...
228         Texas      Wise                73        213            68
229         Texas      Wood               196        893            58
230         Texas     Young                49        153            48
231         Texas    Zapata               140        344            80
232         Texas    Zavala                31         88            20

[233 rows x 5 columns]


In [60]:
#df_torch_texas_nogeo_nostatecounty.shape
# export BRIT Texas summary
brit_texas_county_summary_nogeo.to_csv('TORCH_brit_texas_county_summary_nogeo.csv')

In [155]:
#county_summary_nogeo.to_csv('TORCH_county_summary_nogeo.csv')

In [65]:
# Sampling individual counties
#df_torch_texas_brit_nogeo
# df_torch_texas_brit_nogeo = df_torch_texas_nogeo_nostatecounty[(df_torch_texas_nogeo_nostatecounty['institutionCode'] == 'BRIT') | (df_torch_texas_nogeo_nostatecounty['institutionCode'] == 'VDB')]

df_brit_young_nogeo = df_torch_texas_brit_nogeo[df_torch_texas_brit_nogeo['county'] == 'Young']

df_brit_young_nogeo.shape



(153, 64)

In [66]:
df_brit_young_nogeo.to_csv('brit_young_nogeo.csv')